In [12]:
from mxnet import nd
from mxnet.gluon import nn
#在 Python 的类定义中，所有实例方法的第一个参数都必须是 self
class MLP(nn.Block):
    def __init__(self, **kwargs):
#super(子类，self).__init__(**kwargs)，调用父类中相同的方法
        super(MLP, self).__init__(**kwargs)
        self.hidden = nn.Dense(256, activation = 'relu')
        self.output = nn.Dense(10)
    def forward(self, X):
        return self.output(self.hidden(X))

In [13]:
x = nd.random.uniform(shape=(2,20))
net = MLP()
net.initialize()
net(x)


[[-0.04543568  0.06136357  0.02232315 -0.02003354  0.06826519 -0.04626656
   0.00414387 -0.02961179  0.02218499 -0.03543987]
 [-0.0221923  -0.01521171  0.03840639 -0.01041111  0.07294808 -0.01939896
  -0.00639097 -0.00146172  0.05791141 -0.01838185]]
<NDArray 2x10 @cpu(0)>

In [ ]:
class MySequential(nn.Block):
    def __init__(self, **kwargs):
        super(MySequential, self).__init__(**kwargs)

    def add(self, block):
        # block是一个Block子类实例，假设它有一个独一无二的名字。我们将它保存在Block类的
        # 成员变量_children里，其类型是OrderedDict。当MySequential实例调用
        # initialize函数时，系统会自动对_children里所有成员初始化
        self._children[block.name] = block

    def forward(self, x):
        # OrderedDict保证会按照成员添加时的顺序遍历成员
        for block in self._children.values():
            x = block(x)
        return x

In [47]:
#参数访问
net = nn.Sequential()
net.add(nn.Dense(256, activation = 'relu'))
net.add(nn.Dense(10))
net.initialize()

X = nd.random.uniform(shape= (2,20))
Y = net(X)

for key in net[0].params.keys():
    print(net[0].params[key])

Parameter dense37_weight (shape=(256, 20), dtype=float32)
Parameter dense37_bias (shape=(256,), dtype=float32)


## 参数访问及初始化
- gluon 里的参数用date()访问参数，grad（）访问梯度的数值 
- 指定某层参数初始化, 非首次初始化

    ``` 
    
 net[0].weight,initialize(init=init.Xavier(), force_reinit=True)
 
    ```
- 自定义初始化

```
    class MyInit(init.Initializer):
        def _init_weight(self, name, data):
            print('Init', name, data.shape)
            data[:] = nd.random.uniform(low=-10, high=10, shape=data.shape)
            data *= data.abs() >= 5

    net.initialize(MyInit(), force_reinit=True)
    net[0].weight.data()[0]

```

> 权重形状 = (输出维度, 输入维度) = (out_features, in_features)

## 共享模型参数
疑问：
> 为什么要共享
>  - 减少参数量， 增强稳定性， 编码结构先验（结构先验”就是：提前告诉模型“你大概应该长成什么样，对结构做出假设，结构对称按理说应该提取特征的方式一样，即层参数一样）
> 如何共享（有指定参数）
> ``` 
net = nn.Sequential()
shared = nn.Dense(8, activation = 'relu')
net.add(nn.Dense(8, activation = 'relu'),
      shared,
      nn.Dense(8,activation = 'relu', params = shared.params),
      nn.Dense(10))
net.initialize()
X = nd.random.uniform(shape=(2, 20))
net(X)
net[1].weight.data()[:,1] == net[2].weight.data()[:,1]
>```
> 有什么可能的改进点
> - 实验中验证共享参数的作用
> - 对称金字塔特征提取可以试试

In [54]:
net = nn.Sequential()
shared = nn.Dense(256, activation = 'relu')
net.add(nn.Dense(256,activation = 'relu'),
        nn.Dense(256,activation = 'relu'),
        nn.Dense(20,activation = 'relu', params = shared.params),
        nn.Dense(10))
net.initialize()
train_data = paradas.read_csv()


[1. 1. 1. 1. 1. 1. 1. 1.]
<NDArray 8 @cpu(0)>